In [1]:
from pyspark.sql import SparkSession

In [ ]:
#Initializing a spark Session
spark = SparkSession.builder.appName("debt").getOrCreate()

In [3]:
#Read Csv to DataFrame
debt_df=spark.read.csv("international_debt_with_missing_values.csv", header=True, inferSchema=True) 
debt_df.show(5)

+------------+------------+--------------------+--------------+------------+
|country_name|country_code|      indicator_name|indicator_code|        debt|
+------------+------------+--------------------+--------------+------------+
| Afghanistan|         AFG|Disbursements on ...|DT.DIS.DLXF.CD|7.28944537E7|
| Afghanistan|        NULL|Interest payments...|DT.INT.DLXF.CD|5.32394401E7|
|        NULL|         AFG|PPG, bilateral (A...|DT.AMT.BLAT.CD|6.17393369E7|
| Afghanistan|         AFG|PPG, bilateral (D...|DT.DIS.BLAT.CD|4.91147294E7|
| Afghanistan|         AFG|PPG, bilateral (I...|DT.INT.BLAT.CD|3.99036201E7|
+------------+------------+--------------------+--------------+------------+
only showing top 5 rows


In [4]:
debt_df.printSchema()

root
 |-- country_name: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- indicator_name: string (nullable = true)
 |-- indicator_code: string (nullable = true)
 |-- debt: double (nullable = true)



In [5]:
#Number of rows
debt_df.count()

2357

In [6]:
#Creating a TempView 
debt_df.createOrReplaceTempView("debt_data")

In [7]:
spark.sql("SELECT * FROM debt_data").show(10)

+------------+------------+--------------------+--------------+-------------+
|country_name|country_code|      indicator_name|indicator_code|         debt|
+------------+------------+--------------------+--------------+-------------+
| Afghanistan|         AFG|Disbursements on ...|DT.DIS.DLXF.CD| 7.28944537E7|
| Afghanistan|        NULL|Interest payments...|DT.INT.DLXF.CD| 5.32394401E7|
|        NULL|         AFG|PPG, bilateral (A...|DT.AMT.BLAT.CD| 6.17393369E7|
| Afghanistan|         AFG|PPG, bilateral (D...|DT.DIS.BLAT.CD| 4.91147294E7|
| Afghanistan|         AFG|PPG, bilateral (I...|DT.INT.BLAT.CD| 3.99036201E7|
| Afghanistan|         AFG|PPG, multilateral...|DT.AMT.MLAT.CD|         NULL|
| Afghanistan|         AFG|                NULL|DT.DIS.MLAT.CD| 2.37797243E7|
| Afghanistan|         AFG|                NULL|DT.INT.MLAT.CD|   1.333582E7|
| Afghanistan|         AFG|                NULL|DT.AMT.OFFT.CD|1.008471819E8|
| Afghanistan|         AFG|PPG, official cre...|DT.DIS.OFFT.CD| 

In [8]:

# Replace nulls in numeric columns with 0
debt_df_filled = debt_df.na.fill(0) 
debt_df_filled.show(20)

+------------+------------+--------------------+--------------+-------------------+
|country_name|country_code|      indicator_name|indicator_code|               debt|
+------------+------------+--------------------+--------------+-------------------+
| Afghanistan|         AFG|Disbursements on ...|DT.DIS.DLXF.CD|       7.28944537E7|
| Afghanistan|        NULL|Interest payments...|DT.INT.DLXF.CD|       5.32394401E7|
|        NULL|         AFG|PPG, bilateral (A...|DT.AMT.BLAT.CD|       6.17393369E7|
| Afghanistan|         AFG|PPG, bilateral (D...|DT.DIS.BLAT.CD|       4.91147294E7|
| Afghanistan|         AFG|PPG, bilateral (I...|DT.INT.BLAT.CD|       3.99036201E7|
| Afghanistan|         AFG|PPG, multilateral...|DT.AMT.MLAT.CD|                0.0|
| Afghanistan|         AFG|                NULL|DT.DIS.MLAT.CD|       2.37797243E7|
| Afghanistan|         AFG|                NULL|DT.INT.MLAT.CD|         1.333582E7|
| Afghanistan|         AFG|                NULL|DT.AMT.OFFT.CD|      1.00847

In [9]:
from pyspark.sql import functions as F

In [10]:
from pyspark.sql.functions import col, sum as _sum, avg, countDistinct, count, desc

In [11]:

# 1.  What is the total amount of debt owed by all countries in the dataset?
total_debt = spark.sql("SELECT SUM(debt) AS total_debt FROM debt_data")
total_debt.show()


+-------------------+
|         total_debt|
+-------------------+
|2.82389330025909E12|
+-------------------+



In [12]:
# 2.  How many distinct countries are recorded in the dataset?
total_distinct_countries = spark.sql("SELECT COUNT(DISTINCT country_name) AS total_distinct_countries FROM debt_data")

total_distinct_countries.show()

+------------------------+
|total_distinct_countries|
+------------------------+
|                     124|
+------------------------+



In [13]:
# 3. What are the distinct types of indicators and what do they represent?
distinct_indicators = spark.sql("SELECT DISTINCT indicator_name, indicator_code FROM debt_data")
distinct_indicators.show(20, truncate=False)   # show more rows without truncation


+-------------------------------------------------------------------------------------+--------------+
|indicator_name                                                                       |indicator_code|
+-------------------------------------------------------------------------------------+--------------+
|PPG, commercial banks (INT, current US$)                                             |DT.INT.PCBK.CD|
|PPG, multilateral (AMT, current US$)                                                 |DT.AMT.MLAT.CD|
|Interest payments on external debt, long-term (INT, current US$)                     |DT.INT.DLXF.CD|
|PPG, other private creditors (INT, current US$)                                      |DT.INT.PROP.CD|
|PPG, private creditors (DIS, current US$)                                            |DT.DIS.PRVT.CD|
|PPG, commercial banks (DIS, current US$)                                             |DT.DIS.PCBK.CD|
|Interest payments on external debt, private nonguaranteed (PNG) (INT, cu

In [22]:
## 4. Which country has the highest total debt and how much does it owe?
highest_debt_country = spark.sql("SELECT country_name, SUM(debt) AS total_debt FROM debt_data WHERE country_name IS NOT NULL GROUP BY country_name ORDER BY total_debt DESC LIMIT 10")

highest_debt_country.show()



+--------------------+--------------------+
|        country_name|          total_debt|
+--------------------+--------------------+
|               China|   2.664557603373E11|
|          South Asia|2.436883735975000...|
|              Brazil|   1.761855840362E11|
|  Russian Federation|   1.673876658084E11|
|Least developed c...|1.522543549025000...|
|              Turkey|   1.366056037787E11|
|            IDA only|1.285053249032000...|
|              Mexico|1.233497980602999...|
|               India|1.201164412379000...|
|           Indonesia|    9.25100111052E10|
+--------------------+--------------------+



In [23]:
# 5. What is the average debt across different debt indiactors?
avg_debt_indicators = spark.sql("SELECT indicator_name, AVG(debt) AS avg_debt FROM debt_data GROUP BY indicator_name")
avg_debt_indicators.show(20, truncate=False)


+----------------------------------------------------------------------------------+--------------------+
|indicator_name                                                                    |avg_debt            |
+----------------------------------------------------------------------------------+--------------------+
|Interest payments on external debt, long-term (INT, current US$)                  |1.466122951125743E9 |
|Interest payments on external debt, private nonguaranteed (PNG) (INT, current US$)|7.174920491111112E8 |
|PPG, bilateral (DIS, current US$)                                                 |1.1254369695890102E9|
|PPG, bonds (AMT, current US$)                                                     |1.414863561057143E9 |
|PPG, commercial banks (DIS, current US$)                                          |2.7170178314634144E8|
|NULL                                                                              |1.5384840095781255E9|
|Principal repayments on external debt, long-t

In [24]:
# 6. Which country has made the highest number of principal repayments?
highest_principal_repayments = spark.sql("""
SELECT country_name, COUNT(*) AS repayments_count
    FROM debt_data
    WHERE indicator_name LIKE '%Principal%'
    GROUP BY country_name
    ORDER BY repayments_count DESC
    LIMIT 1
""")
highest_principal_repayments.show()


+------------+----------------+
|country_name|repayments_count|
+------------+----------------+
|        NULL|              16|
+------------+----------------+



In [27]:
# 7. What is the most common debt indicator across all countries?
most_common_indicator = spark.sql("""
    SELECT indicator_name, COUNT(*) AS count
    FROM debt_data
    GROUP BY indicator_name
    ORDER BY count DESC
    LIMIT 1
""")
most_common_indicator.show()


+--------------+-----+
|indicator_name|count|
+--------------+-----+
|          NULL|  250|
+--------------+-----+



In [28]:
# 8. Identify any other key debt trends and summarize your findings
key_trends = spark.sql("""
    SELECT country_name, indicator_name, SUM(debt) AS total_debt
    FROM debt_data
    GROUP BY country_name, indicator_name
    ORDER BY total_debt DESC
    LIMIT 20
""")
key_trends.show(truncate=False)


+--------------------------------------------+-------------------------------------------------------------------------------------+---------------------+
|country_name                                |indicator_name                                                                       |total_debt           |
+--------------------------------------------+-------------------------------------------------------------------------------------+---------------------+
|NULL                                        |Principal repayments on external debt, long-term (AMT, current US$)                  |1.1456974433440001E11|
|China                                       |Principal repayments on external debt, long-term (AMT, current US$)                  |9.62186208357E10     |
|China                                       |Principal repayments on external debt, private nonguaranteed (PNG) (AMT, current US$)|7.23929862138E10     |
|Russian Federation                          |Principal repayments on 